## Import Packages

In [6]:
import struct, csv, json, base64, zlib, pprint, numpy, ctypes
import matplotlib.pyplot as plt
%matplotlib inline

## Relevant Docs

#### [Relevant Python Documentation for Reading C Structs into Python](https://docs.python.org/3.5/library/struct.html "Python Docs for C Structs")

#### [Excellent code snippets for working with Binary data in Python](https://www.devdungeon.com/content/working-binary-data-python "DevDungeon")

|doc | url |
|-----|-----|
| adiBin docs| http://cdn.adinstruments.com/adi-web/manuals/translatebinary/LabChartBinaryFormat.pdf |
| adiBin header|  http://cdn.adinstruments.com/adi-web/manuals/translatebinary/ADIBinaryFormat.h |
| adiBin example|http://cdn.adinstruments.com/adi-web/manuals/translatebinary/TranslateBinary.c |
|source | https://forum.adinstruments.com/viewtopic.php?t=395 |

# Functions

## Base64 to JSON function

In [36]:
def base64ToJson(zippedString):
    json_str = zlib.decompress(base64.b64decode(zippedString)).decode()
    json_json = json.loads(json_str)
    return json_json

## Parse CSV Function

In [41]:
def parseCsv(csv_row, dbg=False):
    
    ############################################################################
    # Parse CSV Data
    ############################################################################
    
    # Record Identifiers 
    patient_id = csv_row[0]
    time_since_admission = csv_row[1]
    
    # Decompress channel data
    channel_json = base64ToJson(row[2])
    
    
    ############################################################################
    # Parse file header and channel header data
    ############################################################################
    
    # Number of channels
    num_channels = len(channel_json)
    
    # Channel titles
    channel_titles = []
    for i in range(num_channels):
        channel_titles.append(channel_json[i]['Label'].upper())
        
    

    ############################################################################
    # Parse channel data
    ############################################################################
    
    # Convert channel data to list
    channel_data = []
    for i in range(num_channels):
        channel_data.append([int(j) for j in channel_json[i]['Text'].split(',')])
        
    
    # Check to see if all channels have the same length
    # If channels are the same length, samples_per_channel set to len element 0
    # If theres a mismatch, pad other channels with mean and
    #   set samples_per_channel to max length
    
    if all(len(i) == len(channel_data[0]) for i in channel_data):
        samples_per_channel = len(channel_data[0])
        if dbg == True:
            print("all channel_data sublists equal length")
    else:
        max_sublist_len = len(max(channel_data, key=len))
        for i in range (num_channels):
            len_dif_tuple = (0, max_sublist_len - len(channel_data[i]))
            channel_data[i] = numpy.pad(channel_data[i]\
                                        , pad_width = len_dif_tuple\
                                        , mode='mean'
                                       )
        samples_per_channel = max_sublist_len  
        if dbg == True:
            print("all channel_data sublists NOT equal length \
                  - padded w/ trailing means")


    ############################################################################
    # Create dictionary of all parsed data from csv file
    ############################################################################
    
    csv_data = {
          'patient_id' : patient_id
        , 'time_since_admission' : time_since_admission
        , 'num_channels' : num_channels
        , 'channel_titles' : channel_titles
        , 'channel_data' : channel_data
        , 'samples_per_channel' : samples_per_channel
    }
    
    
    ############################################################################
    # Debug: Check values
    ############################################################################
    if dbg == True:
        print("patient_id:", patient_id)
        print("time_since_admission:", time_since_admission)
        print("num_channels:", num_channels)
        print("channel_titles:", channel_titles)
        print("samples_per_channel:", samples_per_channel)
        print("channel_data len:", len(channel_data))
        print("channel_data[0] len:", len(channel_data[0]))
        print("\n")
        #print("channel_data[0]:", channel_data[0])
        
    
    ############################################################################
    # Return
    ############################################################################
    return csv_data

## Create ADIBIN Function

In [50]:
def createAdibin(data_dict, dbg=False):
    
    ############################################################################
    # Define default adibin file header variables
    ############################################################################
    
    # Define default file size variables for writing adibin file
    FILE_HEADER_LENGTH = 68
    CHANNEL_HEADER_LENGTH = 96
    CHANNEL_TITLE_LENGTH = 32
    UNITS_LENGTH = 32
    
    # Define format strings for struct
    ADI_FILE_HEADER_FORMAT_STRING = "<4sldlllllddllll"
    ADI_CHANNEL_HEADER_FORMAT_STRING = "<32s32sdddd"
    
    # Define default file header variables
    magic = b'CFWB'
    version = 1
    secs_per_tick = 1/240
    year = 1776
    month = 7
    day = 4
    hour = 0
    minute = 0
    second = 0
    trigger = 0
    # num_channels passed in dict
    # samples_per_channel passed in dict
    time_channel = 0
    data_format = 3
    
    # Define default channel header variables
    # channel_title passed in dict
    units = {'I':b'mV' \
           , 'II':b'mV' \
           , 'III':b'mV' \
           , 'V':b'mV' \
           , 'AVR':b'mV' \
           , 'AVL':b'mV' \
           , 'AVF':b'mV'  \
           , 'AR2':b'mmHg' \
           , 'SPO2':b'%' \
           , 'RR':b'Imp' \
           , 'RESP':b'Imp'\
           }
    scale = {'I': 2.44 \
           , 'II': 2.44 \
           , 'III': 2.44 \
           , 'V': 2.44 \
           , 'AVR': 2.44 \
           , 'AVL': 2.44 \
           , 'AVF': 2.44  \
           , 'AR2': 0.2 \
           , 'SPO2': 1.0 \
           , 'RR': 0.1 \
           , 'RESP': 0.1 \
           }
    offset = 0
    range_high = 1
    range_low = 0
    # channel_data passed in dict
    
    
    ############################################################################
    # Create units and scales fields from channel titles  passed in dict
    ############################################################################
    
    # Channel units
    channel_units = []
    for i in range(data_dict['num_channels']):
        try:
            channel_units.append(units[data_dict['channel_titles'][i]])
        except:
            channel_units.append('MysteryChannelUnits')
        
    # Channel scales
    channel_scales = []
    for i in range(data_dict['num_channels']):
        try:
            channel_scales.append(scale[data_dict['channel_titles'][i]])
        except:
            channel_scales.append('1.0')
            
   
    ############################################################################
    # Convert channel titles passed in dict to binary strings
    ############################################################################
    
    # New list for binary channel titles
    bin_channel_titles = []
    for i in range(data_dict['num_channels']):
        bin_channel_titles.append(\
                                data_dict['channel_titles'][i].encode('utf-8'))
    
    ############################################################################
    # Append all adibin fields to list of lists
    ############################################################################
    
    # Append file header    
    adibin_header = []
    
    adibin_header.append(magic)
    adibin_header.append(version)
    adibin_header.append(secs_per_tick)
    adibin_header.append(year)
    adibin_header.append(month)
    adibin_header.append(day)
    adibin_header.append(hour)
    adibin_header.append(minute)
    adibin_header.append(second)
    adibin_header.append(trigger)
    adibin_header.append(data_dict['num_channels'])
    adibin_header.append(data_dict['samples_per_channel'])
    adibin_header.append(time_channel)
    adibin_header.append(data_format)
 
    # Append channel headers
    adibin_channel_headers = []

    for i in range(data_dict['num_channels']):
        adibin_channel_headers.append(bin_channel_titles[i])
        adibin_channel_headers.append(channel_units[i])
        adibin_channel_headers.append(channel_scales[i])
        adibin_channel_headers.append(offset)
        adibin_channel_headers.append(range_high)
        adibin_channel_headers.append(range_low)
        
    # Append interleaved channel data
    adibin_channel_data = []
    
    for j in range(data_dict['samples_per_channel']):
        for i in range(data_dict['num_channels']):
            adibin_channel_data.append(data_dict['channel_data'][i][j])
        
    
    
    ############################################################################
    # Pack adibin data into writable struct
    ############################################################################
    
    # Pack file header
    file_header_buffer = bytearray(FILE_HEADER_LENGTH)
    struct.pack_into(
        ADI_FILE_HEADER_FORMAT_STRING
        , file_header_buffer
        , 0
        , *adibin_header
        )

    # Pack channel headers
    channel_headers_buffer = bytearray((data_dict['num_channels']) \
                                       * CHANNEL_HEADER_LENGTH)
    
    channel_headers_format_string = ADI_CHANNEL_HEADER_FORMAT_STRING
    for i in range(data_dict['num_channels'] - 1):
        channel_headers_format_string += ADI_CHANNEL_HEADER_FORMAT_STRING[1:]
    
    struct.pack_into(
        channel_headers_format_string
        , channel_headers_buffer
        , 0
        , *adibin_channel_headers
        )

    
    ############################################################################
    # Debug: Check values
    ############################################################################
    if dbg == True:
        print("channel_units:", channel_units)
        print("channel_scales:", channel_scales)
        #print('adibin:', adibin)
        print("file_header_buffer:", file_header_buffer)
        print('\n')
        
        
    ############################################################################
    # Return
    ############################################################################
    return True

## Sanity Check

In [51]:
csvFileName = "./sampleFiles/4b80ff2eb1112815299d7a4e9a4a1957.csv"

csvFile = open(csvFileName)
csvFileReader = csv.reader(csvFile)

for row in csvFileReader:
    createAdibin(parseCsv(row, dbg=True), dbg=True)
    print('-----')

all channel_data sublists equal length
patient_id: ea6392983e1786204d56da60f29cb2dc
time_since_admission: 41842
num_channels: 9
channel_titles: ['I', 'II', 'III', 'V', 'AVR', 'AVL', 'AVF', 'SPO2', 'RESP']
samples_per_channel: 2400
channel_data len: 9
channel_data[0] len: 2400


channel_units: [b'mV', b'mV', b'mV', b'mV', b'mV', b'mV', b'mV', b'%', b'Imp']
channel_scales: [2.44, 2.44, 2.44, 2.44, 2.44, 2.44, 2.44, 1.0, 0.1]
file_header_buffer: bytearray(b'CFWB\x01\x00\x00\x00\x11\x11\x11\x11\x11\x11q?\xf0\x06\x00\x00\x07\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\t\x00\x00\x00`\t\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00')


-----
all channel_data sublists equal length
patient_id: 501d5ed00964868b6115eb924681851e
time_since_admission: 46596
num_channels: 9
channel_titles: ['I', 'II', 'III', 'V', 'AVR', 'AVL', 'AVF', 'SPO2', 'RESP']
samples_per_channel: 2400
channel_data len: 9
channel_data[0] len: 2400


chan